### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220921_scenario_projections.ipynb")

2022/09/20 22:41:12 INFO|============================================================


### Fit an initial scenario through February 2022

In [12]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-12-31',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/22 15:54:21 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/22 15:54:24 INFO|-------------------------|{} Constructing vaccination projections
2022/09/22 15:54:24 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/22 15:54:27 INFO|----------------------|{} Prepping Model
2022/09/22 15:54:27 INFO|-----------------------|{} Prepping Model
2022/09/22 15:55:37 INFO|----------------------|{} Model prepped for fitting in 70.3231437089853 seconds.
2022/09/22 15:55:37 INFO|----------------------|{} Will fit 25 times
2022/09/22 15:56:44 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 66.63883554199128 seconds: {0: {'co': 0.8587936471077695}, 14: {'co': 0.6897060210057149}, 28: {'co': 0.5521354803684264}, 42: {'co': 0.0012797390762758045}, 56: {'co': 0.6815578687612924}, 70: {'co': 0.891229495822175}}
2022/09/22 15:56:44 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/22

2022/09/22 16:02:48 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 21.44337262501358 seconds: {504: {'co': 0.7861683694318742}, 518: {'co': 0.7799072714975074}, 532: {'co': 0.7570400029133451}, 546: {'co': 0.7091683976662693}, 560: {'co': 0.7430699239544509}, 574: {'co': 0.7376116902266164}}
2022/09/22 16:02:48 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/22 16:03:23 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 33.89440612500766 seconds: {532: {'co': 0.7562144511906196}, 546: {'co': 0.7112539505348578}, 560: {'co': 0.7393425217824591}, 574: {'co': 0.7492676796246258}, 588: {'co': 0.786256629101155}, 602: {'co': 0.7652674658065619}}
2022/09/22 16:03:23 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/22 16:03:50 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 25.013901291997172 seconds: {5

### Create and run scenarios from Feb 2022 to present

In [13]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20220921_sep_gov_briefing/20220921_scenario_params.json"))

model_args = {
    'base_spec_id': 4011, #model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 2]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
                   "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-20": vx_seed, "2022-10-20": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                          "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                          "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                          "2022-08-20": 0.945 + 0.055*vacc_eff_lt5,
                                          "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                          "2022-09-10": 0.935 + 0.065*vacc_eff_lt5,
                                          "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                                "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'vx_seed': vx_seed,
                                                 'vir_mult': vir_mult}})
            

In [14]:
# check how many scenarios there are
len(scenario_model_args)

4

In [15]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/22 16:35:15 INFO|============================================================
2022/09/22 16:35:15 INFO|============================================================
2022/09/22 16:35:15 INFO|============================================================
2022/09/22 16:35:15 INFO|============================================================
2022/09/22 16:35:18 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/22 16:35:18 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/22 16:35:18 INFO|-----------|{'vx_seed': 2, 'vir_mult': 0.833} Retrieving vaccinations data
2022/09/22 16:35:18 INFO|-----------|{'vx_seed': 2, 'vir_mult': 2.38} Retrieving vaccinations data
2022/09/22 16:35:25 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38} Constructing vaccination projections
2022/09/22 16:35:25 INFO|-----------|{'vx_seed': 2, 'vir_mult': 2.38} Constructing vaccination projections
2022/09/22 16:35:25 INFO|-----------|{'vx_se

2022/09/22 16:37:26 INFO|--------|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 13.08439887499999 seconds: {823: {'co': 0.8472556501976408}, 837: {'co': 0.8452670140619504}, 851: {'co': 0.8317732054941022}, 865: {'co': 0.8610008595566302}, 879: {'co': 0.8527016329807836}, 893: {'co': 0.8569091813153676}}
2022/09/22 16:37:26 INFO|---------|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '2'}: Running forward sim
2022/09/22 16:37:40 INFO|--------|{'vx_seed': 2, 'vir_mult': 2.38, 'fit_batch': '3'}: Transmission control fit 4/5 completed in 13.179937291999977 seconds: {851: {'co': 0.8320791591261223}, 865: {'co': 0.860567048164575}, 879: {'co': 0.8536951995138884}, 893: {'co': 0.854080567568037}, 907: {'co': 0.8616389507142609}, 921: {'co': 0.8574376250218094}}
2022/09/22 16:37:40 INFO|---------|{'vx_seed': 2, 'vir_mult': 2.38, 'fit_batch': '3'}: Running forward sim
2022/09/22 16:37:40 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch

2022/09/22 16:38:00 INFO|--------|{'vx_seed': 2, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8587936461316271}, 14: {'co': 0.6897060243996839}, 28: {'co': 0.5462679583870651}, 42: {'co': 1.451583333544111e-14}, 56: {'co': 0.6964653973176903}, 70: {'co': 0.8333062151413904}, 84: {'co': 0.8295950198988893}, 98: {'co': 0.8317180241258532}, 112: {'co': 0.8774454884904278}, 126: {'co': 0.8243715380856421}, 140: {'co': 0.7604769068144684}, 154: {'co': 0.671254142980569}, 168: {'co': 0.8430274770525327}, 182: {'co': 0.7826503986321877}, 196: {'co': 0.8191652894252891}, 210: {'co': 0.7726023547909344}, 224: {'co': 0.7713430213084852}, 238: {'co': 0.7185520643368669}, 252: {'co': 0.7221395037804547}, 266: {'co': 0.6850199020453478}, 280: {'co': 0.7198410199610565}, 294: {'co': 0.7578493571436148}, 308: {'co': 0.8225927200337738}, 322: {'co': 0.8115175648591666}, 336: {'co': 0.7825629134751998}, 350: {'co': 0.795387566558624}, 364: {'co': 0.8054779547279171},

### Run the report for each fit model

In [11]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir,
                           prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/09/22 15:27:48 INFO|============================================================
2022/09/22 15:27:48 INFO|--------|Solving model
2022/09/22 15:27:49 INFO|============================================================
2022/09/22 15:27:49 INFO|--------|Solving model
2022/09/22 15:27:49 INFO|============================================================
2022/09/22 15:27:49 INFO|--------|Solving model
2022/09/22 15:27:50 INFO|============================================================
2022/09/22 15:27:50 INFO|--------|Solving model
2022/09/22 15:27:53 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/22 15:27:53 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/22 15:27:54 WARNING|No artists with labels found to put in legend.  Note that artists 

In [16]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-12-31', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-12-31', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/22 16:45:39 INFO|Projecting
2022/09/22 16:45:39 INFO|
2022/09/22 16:45:55 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/22 16:45:57 INFO|
2022/09/22 16:46:13 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/22 16:46:14 INFO|
2022/09/22 16:46:31 INFO|{'vx_seed': 2, 'vir_mult': 0.833, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/22 16:46:33 INFO|
2022/09/22 16:46:50 INFO|{'vx_seed': 2, 'vir_mult': 2.38, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/22 16:46:51 INFO|Running reports
